### Hybrid recall

How might the modern Hopfield network and generative model work together to recall a partially consolidated memory?

#### Installation:

In [ ]:
!pip install -r requirements.txt

#### Imports:

In [ ]:
from end_to_end import run_end_to_end
import tensorflow as tf
from utils import prepare_data, noise, display
from initial_model import create_autoencoder
from initial_tests import check_initial_recall, iterative_recall
from generative_model import VAE, build_encoder_decoder_v3, build_encoder_decoder_v5
from generative_tests import interpolate_ims, plot_latent_space, check_generative_recall, plot_history, vector_arithmetic
from tensorflow import keras
import numpy as np
from random import randrange
from PIL import Image
import matplotlib.pyplot as plt
import hopfield_utils
from hopfield_models import ContinuousHopfield
import matplotlib.backends.backend_pdf
from config import models_dict, dims_dict

# set tensorflow random seed to make outputs reproducible
tf.random.set_seed(1)

In [ ]:
def run_end_to_end(dataset='mnist', initial_epochs=10, generative_epochs=10, num=100, latent_dim=5, kl_weighting=1, hopfield_type='continuous', lr=0.001):
    
    train_data, test_data, noisy_train_data, noisy_test_data = prepare_data(dataset)
    
    dims = dims_dict[dataset]
    model = models_dict[dataset]
        
    net = hopfield_utils.create_hopfield(num, hopfield_type=hopfield_type, dataset=dataset)
    predictions = []
    tests = []

    images = hopfield_utils.load_images_mnist(num, dataset=dataset)

    images_np = hopfield_utils.convert_images(images, 'gray-scale')
    images_masked_np = hopfield_utils.mask_image_random(images_np)
    images_np = [im_np.reshape(-1, 1) for im_np in images_np]
    images_masked_np = [im_np.reshape(-1, 1) for im_np in images_masked_np]

    for test_ind in range(num):
        test = images_masked_np[test_ind].reshape(-1,1)
        if hopfield_type is 'classical':
            reconstructed = net.retrieve(test)
        else:
            reconstructed = net.retrieve(test, max_iter=10)
        # if your image is greyscale, you need to pass PIL 2d array
        reshaped = np.array(reconstructed).reshape(1,dims[0],dims[1])
        test = np.array(test).reshape(1,dims[0],dims[1])
        predictions.append(reshaped.reshape(1,dims[0],dims[1],dims[2]))
        tests.append(test.reshape(1,dims[0],dims[1],dims[2]))

    predictions = np.concatenate(predictions, axis=0)
    tests = np.concatenate(tests, axis=0)

    # rescale predictions back to interval [0, 1]
    predictions = (predictions + 1) / 2 

    encoder, decoder = models_dict[dataset](latent_dim = latent_dim)
    vae = VAE(encoder, decoder, kl_weighting)
    opt = keras.optimizers.Adam(lr=lr)
    vae.compile(optimizer=opt)
    history = vae.fit(predictions, epochs=generative_epochs, verbose=0, batch_size=32, shuffle=True)

    return net, vae

def check_generative_recall(vae, test_data, noise_level=0.25):
    test_data = noise(test_data, noise_factor=noise_level)
    latents = vae.encoder.predict(test_data)
    predictions = vae.decoder.predict(latents[0])
    fig = display(test_data, predictions, title='Inputs and outputs for VAE')
    return predictions
        

The cell below recreates the results in the 'outputs' folder:

In [ ]:
net, vae = run_end_to_end(dataset='mnist', generative_epochs=1000, 
               num=3000, latent_dim=10, kl_weighting=1)

#### Hybrid recall

First, pass noisy images through the partially-trained VAE, producing an approximate version of a memory. Then give the VAE output to the modern Hopfield network to restore the details.

In [ ]:
train_data, test_data, noisy_train_data, noisy_test_data = prepare_data('mnist')
preds = check_generative_recall(vae, train_data[0:10], noise_level=0.25)

# Rescale to range -1 to 1 for Hopfield network:
preds = (preds * 2) - 1

In [ ]:
def display(array1, array2, seed=None, title='Inputs and outputs of the model', dataset='mnist'):
    hopfield=False
    
    dim = array1[0].shape[0]
    #Displays ten random images from each one of the supplied arrays.
    if seed is not None:
        np.random.seed(seed)
        
    n = 10

    images1 = array1
    images2 = array2

    fig = plt.figure(figsize=(20, 4))
    for i, (image1, image2) in enumerate(zip(images1, images2)):
        ax = plt.subplot(2, n, i + 1)
        if hopfield==True:
            plt.imshow(image1.reshape(dim, dim), cmap='binary', vmin=-1, vmax=1)
        else:
            plt.imshow(image1.reshape(dim, dim))
            plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, n, i + 1 + n)
        if hopfield==True:
            plt.imshow(image2.reshape(dim, dim), cmap='binary', vmin=-1, vmax=1)
        else:
            plt.imshow(image2.reshape(dim, dim))
            plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    
    fig.suptitle(title)
    plt.show()
    return fig

In [ ]:
num = 10
dims=(28,28,1)

predictions = []
tests = []

for test_ind in range(num):
    test = preds[test_ind].reshape(-1,1)
    reconstructed = net.retrieve(test, max_iter=10)
    # if your image is greyscale, you need to pass PIL 2d array
    reshaped = np.array(reconstructed).reshape(1,dims[0],dims[1])
    test = np.array(test).reshape(1,dims[0],dims[1])
    predictions.append(reshaped.reshape(1,dims[0],dims[1],dims[2]))
    tests.append(test.reshape(1,dims[0],dims[1],dims[2]))

print("Recalling noisy images with the initial model:")
predictions = np.concatenate(predictions, axis=0)
tests = np.concatenate(tests, axis=0)

fig = display(tests,predictions, dataset='mnist', title='Inputs and outputs for Hopfield network')      